Objectives

1) how to log in to the ArcGIS Online and NOAA GeoPlatform
2) create a basic map
3) search for and add ArcGIS Online or NOAA GeoPlatform layers (eclipse path, eclipse cities)
4) download netcdf file from NCAR Thredds server (bonus if you can do all this from python)
Datset page:
https://thredds.ucar.edu/thredds/ncss/grid/grib/NCEP/NBM/CONUS/National_Blend_CONUS_20240405_1600.grib2/dataset.html
-select variable "Total cloud cover @ Ground or water surface"
-set Time Subset to "single time" and value "2024-04-08T18:00:00Z"
-set Download format to "netcdf4-classic"

Final Download URL:
https://thredds.ucar.edu/thredds/ncss/grid/grib/NCEP/NBM/CONUS/National_Blend_CONUS_20240405_1600.grib2?var=Total_cloud_cover_surface&north=57.098&west=-138.394&east=-59.022&south=19.215&horizStride=1&time=2024-04-08T18:00:00Z&&accept=netcdf4-classic
5) Upload to your JupyterHub file home
6) (in external geospatial python container) read National Blend of Models (NBM) GRIB2 sky cover data, select cells with sky cover less or equal to 25%, convert raster to vector polygons, export to file
7) Start jupyterhub with a the custom arcgis python image
- https://itcoocean.2i2c.cloud/
- log in
- bring your own image
- 
8) read file into feature collection, display on map
9) find cities that are within the good sky area
10) select a city, create a drive time area of 3 hours
11) Find cities outside the drive area



In [1]:
from arcgis.gis import GIS

Let's create a gis connection to the public ArcGIS Online

In [2]:
agol = GIS()

In [3]:
m = agol.map()
m

MapView(layout=Layout(height='400px', width='100%'))

In [4]:
content = agol.content.search(query="title: eclipse AND owner:NWS.FWD_noaa", 
                                item_type="Feature Layer", 
                                max_items=15)

In [19]:
content = agol.content.search(query="id:80f6cd4b89734ec1adb541a789879727", 
                                item_type="Feature Layer", 
                                max_items=15)

In [20]:
content

[<Item title:"BLM NV Annular Solar Eclipse Duration Lines" type:Feature Layer Collection owner:s76moore@blm.gov_BLM_EGIS>]

In [6]:
item = content[1]

In [7]:
item

<Item title:"2024_Eclipse_Shapefiles" type:Feature Layer Collection owner:NWS.FWD_noaa>

In [8]:
item.layers

[<FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/0">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/1">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/2">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/3">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/4">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/5">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/2024_Eclipse_Shapefiles/FeatureServer/6">,
 <FeatureLayer url:"https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/202

In [9]:
item.layers[0].properties

{
  "currentVersion": 11.2,
  "id": 0,
  "name": "center",
  "type": "Feature Layer",
  "serviceItemId": "12d5ada7da474b0cb678a3b067e67011",
  "cacheMaxAge": 30,
  "displayField": "",
  "description": "",
  "copyrightText": "",
  "defaultVisibility": true,
  "editingInfo": {
    "lastEditDate": 1708267630164,
    "schemaLastEditDate": 1708267630164,
    "dataLastEditDate": 1708267630164
  },
  "relationships": [],
  "isDataVersioned": false,
  "hasContingentValuesDefinition": false,
  "supportsAppend": true,
  "supportsCalculate": true,
  "supportsASyncCalculate": true,
  "supportsTruncate": true,
  "supportsAttachmentsByUploadId": true,
  "supportsAttachmentsResizing": true,
  "supportsRollbackOnFailureParameter": true,
  "supportsStatistics": true,
  "supportsExceedsLimitStatistics": true,
  "supportsAdvancedQueries": true,
  "supportsValidateSql": true,
  "supportsCoordinatesQuantization": true,
  "supportsLayerOverrides": true,
  "supportsTilesAndBasicQueriesMode": true,
  "support

In [10]:
i = 0
for l in item.layers:
    print(i, l.properties["name"])
    i = i + 1

0 center
1 duration
2 ppath
3 ppath01
4 umbra_hi
5 umbra_lo
6 upath_hi
7 upath_lo


In [11]:
center_layer = item.layers[0]

In [12]:
m.add_layer(center_layer, options={'opacity':0.3})

In [13]:
m

MapView(jupyter_target='lab', layout=Layout(height='400px', width='100%'), ready=True)

In [14]:
ppath_layer = item.layers[2]

In [15]:
m.add_layer(ppath_layer, options={'opacity':0.3})

In [16]:
m.remove_layers(layers=[ppath_layer])

True

In [17]:
upath_lo_layer = item.layers[7]

In [18]:
m.add_layer(upath_lo_layer, options={'opacity':0.3})

In order to log in to the NOAA GeoPlatform, you need to create a "client_id" 

In [21]:
gis = GIS("https://noaa.maps.arcgis.com",client_id="sidPYKs786TiwBcz")

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://noaa.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=sidPYKs786TiwBcz&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=AHQXYrViZgxeyGsgFCYFBR9F6fNZ11


Enter code obtained on signing in using SAML:  ············································································································································································································································


/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'noaa.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [22]:
print("Successfully logged in as: " + gis.properties.user.username)

Successfully logged in as: Tim.Haverland_noaa


Geocode a point

In [20]:
from arcgis.geocoding import geocode

In [21]:
results = geocode('Burlington, VT',as_featureset=True)

In [22]:
results

<FeatureSet> 3 features

In [23]:
results.features

[{"geometry": {"x": -73.2109954, "y": 44.4759918, "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"Loc_name": "World", "Status": "T", "Score": 100, "Match_addr": "Burlington, Vermont", "LongLabel": "Burlington, VT, USA", "ShortLabel": "Burlington", "Addr_type": "Locality", "Type": "City", "PlaceName": "Burlington", "Place_addr": "Burlington, Vermont", "Phone": "", "URL": "", "Rank": 8.69, "AddBldg": "", "AddNum": "", "AddNumFrom": "", "AddNumTo": "", "AddRange": "", "Side": "", "StPreDir": "", "StPreType": "", "StName": "", "StType": "", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "", "Block": "", "Sector": "", "Nbrhd": "", "District": "", "City": "Burlington", "MetroArea": "", "Subregion": "Chittenden County", "Region": "Vermont", "RegionAbbr": "VT", "Territory": "", "Zone": "", "Postal": "", "PostalExt": "", "Country": "USA", "CntryName": "United States", "LangCode": "E

In [24]:
results.features.pop(0)

{"geometry": {"x": -73.2109954, "y": 44.4759918, "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"Loc_name": "World", "Status": "T", "Score": 100, "Match_addr": "Burlington, Vermont", "LongLabel": "Burlington, VT, USA", "ShortLabel": "Burlington", "Addr_type": "Locality", "Type": "City", "PlaceName": "Burlington", "Place_addr": "Burlington, Vermont", "Phone": "", "URL": "", "Rank": 8.69, "AddBldg": "", "AddNum": "", "AddNumFrom": "", "AddNumTo": "", "AddRange": "", "Side": "", "StPreDir": "", "StPreType": "", "StName": "", "StType": "", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "", "Block": "", "Sector": "", "Nbrhd": "", "District": "", "City": "Burlington", "MetroArea": "", "Subregion": "Chittenden County", "Region": "Vermont", "RegionAbbr": "VT", "Territory": "", "Zone": "", "Postal": "", "PostalExt": "", "Country": "USA", "CntryName": "United States", "LangCode": "EN

In [25]:
results.features.pop(0)

{"geometry": {"x": -73.21207, "y": 44.47588, "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"Loc_name": "World", "Status": "T", "Score": 100, "Match_addr": "Burlington, Vermont", "LongLabel": "Burlington, VT, USA", "ShortLabel": "Burlington", "Addr_type": "Locality", "Type": "City", "PlaceName": "Burlington", "Place_addr": "Burlington, Vermont", "Phone": "", "URL": "", "Rank": 13.72, "AddBldg": "", "AddNum": "", "AddNumFrom": "", "AddNumTo": "", "AddRange": "", "Side": "", "StPreDir": "", "StPreType": "", "StName": "", "StType": "", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "", "Block": "", "Sector": "", "Nbrhd": "", "District": "", "City": "Burlington", "MetroArea": "", "Subregion": "Chittenden County", "Region": "Vermont", "RegionAbbr": "VT", "Territory": "", "Zone": "", "Postal": "", "PostalExt": "", "Country": "USA", "CntryName": "United States", "LangCode": "ENG",

In [26]:
m.draw(results)

Create Drive Time Area

In [27]:
import arcgis.network as network

In [28]:
service_area_url = gis.properties.helperServices.serviceArea.url

In [29]:
service_area_url

'https://route.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World'

In [30]:
sa_layer = network.ServiceAreaLayer(service_area_url, gis=gis)

In [31]:
result = sa_layer.solve_service_area(results, default_breaks=[180], 
                                     travel_direction='esriNATravelDirectionFromFacility')

In [32]:
from arcgis.features import Feature, FeatureSet

In [33]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)

In [34]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

In [35]:
m.draw(service_area_fset)